In [1]:
import pandas as pd
import netpixi
from graph_tool import Graph
from netpixi.data.gt import *
from graph_tool import centrality
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv("data.csv")

In [3]:
df_artista = df.artists

In [4]:
mask = (df['release_date'] > "1990-01-01") & (df['release_date'] <= "2020-05-15")

In [5]:
df_recorte = df.loc[mask]

In [6]:
artistas = df_recorte['artists'].tolist()

In [7]:
flat_list = []
lista_artistas_musicas = []
for sublist in artistas:
    sub = sublist.strip(" ] [ ").split(", ")
    lista_artistas_musicas.append(sub)
    for e in sub:
        flat_list.append(e.replace("'",""))

In [8]:
artistas_certo = []
sub_sub = []
for i in lista_artistas_musicas:
    sub_sub = []
    for e in i:
        e = e.replace("'","")
        sub_sub.append(e.strip('""'))
    artistas_certo.append(sub_sub)

In [9]:
df_recorte['artists'] = artistas_certo

In [10]:
total_artista = set(flat_list)


## criando dicionário com todos os artistas e sua popularidade média de músicas

In [11]:
pop = df_recorte["popularity"].tolist()

In [12]:
count = 0
pop_new = []
for a in artistas_certo:
    for g in a:
        pop_new.append(pop[count])
    count += 1

In [13]:
pop_musica_artista = pd.DataFrame({
    "artista":flat_list,
    "popularidade": pop_new
})

In [14]:
pop_musica_artista

,artista,popularidade
0,Paul Simon,35
1,Concrete Blonde,35
2,Roberta Flack,35
3,Donny Hathaway,35
4,Benny Hinn,37
...,...,...
75538,DripReport,75
75539,Tyga,75
75540,Cash Cash,70
75541,Andy Grammer,70


In [15]:
df_artista_media_pop = pop_musica_artista.groupby("artista").mean().reset_index()

In [16]:
df_artista_media_pop.artista = df_artista_media_pop.artista.str.strip('""')

In [17]:
df_artista_media_pop

,artista,popularidade
0,Adolescents Orquesta,60.111111
1,Alexander ONeal,45.333333
2,Amy Hanaialii,37.000000
3,Aulii Cravalho,66.800000
4,B. DAddario as Flounder,44.000000
...,...,...
15566,辛曉琪,46.500000
15567,那英,46.500000
15568,高慧君,52.000000
15569,黃品源,47.000000


In [18]:
musicas_com_feat = []
for e in artistas_certo:
    if len(e) > 1:
        musicas_com_feat.append(e)

In [19]:
musicas_com_feat_flat = [item.strip('""') for sublist in musicas_com_feat for item in sublist]


In [20]:
artista_participantes_feat = list(set(musicas_com_feat_flat))

### Criando rede

In [21]:
g = Graph(directed=False)
g.vp['Nome'] = g.new_vp('object')
g.vp['Pop_media'] = g.new_vp('object')
g.ep["Numero_parcerias"] = g.new_ep("object")

### Criando nós só com os artistas que fizeram colaborações

In [22]:
counter = 0
for art in range(0,len(artista_participantes_feat)):
    g.add_vertex()
    g.vp["Nome"][art] = artista_participantes_feat[art]
    g.vp["Pop_media"][art] = df_artista_media_pop.loc \
    [df_artista_media_pop["artista"] == artista_participantes_feat[art],"popularidade"].iloc[0]
    counter += 1

In [23]:
dic = {}
for artista in artista_participantes_feat:
    dic[artista] = {}
    for musica in musicas_com_feat:
        if artista in musica:
            for participante in musica:
                if artista != participante:
                    #dic[artista].append(participante)
                    if participante not in dic[artista]:
                        dic[artista][participante] = 1
                    else:
                        dic[artista][participante] += 1
                 

In [24]:
contador = 0
for artista, participantes in dic.items():
    for participante, ocorrencia in participantes.items():
        no1 = artista_participantes_feat.index(artista)
        no2 = artista_participantes_feat.index(participante)
        g.add_edge(no1,no2)
        contador += 1

### Criando nós com os artistas que não participaram de feat

In [25]:
artista_nao_colaboradores = []
for art in total_artista:
    art = art.strip('""')
    if art not in musicas_com_feat_flat:
        artista_nao_colaboradores.append(art)

In [26]:
for art in range(0,len(artista_nao_colaboradores)):
    g.add_vertex()
    g.vp["Nome"][counter] = artista_nao_colaboradores[art]
    g.vp["Pop_media"][counter] = df_artista_media_pop.loc \
    [df_artista_media_pop["artista"] == artista_nao_colaboradores[art],"popularidade"].iloc[0]
    counter += 1

In [27]:
dc = degree_gt(g)
g.vp['degree'] = dc

bc, _ = centrality.betweenness(g)
g.vp['betweenness'] = bc

cc = centrality.closeness(g)
g.vp['closeness'] = cc

_, ec = centrality.eigenvector(g)
g.vp['eigenvector'] = ec

In [28]:
lista_completa = artista_participantes_feat + artista_nao_colaboradores

In [29]:
df_completo = pd.DataFrame({
    "artista":lista_completa
})

In [30]:
df_completo = pd.DataFrame({k: list(v) for k, v in g.vp.items()})

In [53]:
df_completo = df_completo.fillna(0)

In [54]:
import statsmodels.formula.api as smf

model = smf.ols(data=df_completo, formula='Pop_media ~ degree') # z1 é explicada por x1?
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              Pop_media   R-squared:                       0.010
Model:                            OLS   Adj. R-squared:                  0.010
Method:                 Least Squares   F-statistic:                     158.8
Date:                Mon, 08 Nov 2021   Prob (F-statistic):           3.12e-36
Time:                        14:09:38   Log-Likelihood:                -57947.
No. Observations:               15570   AIC:                         1.159e+05
Df Residuals:                   15568   BIC:                         1.159e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     50.6288      0.086    589.566      0.000      50.461      50.797
degree      1253.7556     99.494     12.601      0.000    1058.735    1448.776
==============================================================================
Omnibus:                      387.430   Durbin-Watson:                   1.986
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              395.388
Skew:                           0.369   Prob(JB):                     1.39e-86
Kurtosis:                       2.744   Cond. No.                     1.24e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.24e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [55]:
model2 = smf.ols(data=df_completo, formula='Pop_media ~ betweenness ') # z1 é explicada por x1?
result2 = model2.fit()
result2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              Pop_media   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     27.23
Date:                Mon, 08 Nov 2021   Prob (F-statistic):           1.83e-07
Time:                        14:09:39   Log-Likelihood:                -58012.
No. Observations:               15570   AIC:                         1.160e+05
Df Residuals:                   15568   BIC:                         1.160e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      50.9624      0.081    627.814      0.000      50.803      51.122
betweenness  1424.3515    272.957      5.218      0.000     889.324    1959.379
==============================================================================
Omnibus:                      377.817   Durbin-Watson:                   1.969
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              385.299
Skew:                           0.364   Prob(JB):                     2.15e-84
Kurtosis:                       2.747   Cond. No.                     3.39e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.39e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [56]:
model3 = smf.ols(data=df_recorte, formula='popularity ~ energy ') # z1 é explicada por x1?
result3 = model3.fit()
result3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             popularity   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     53.25
Date:                Mon, 08 Nov 2021   Prob (F-statistic):           2.97e-13
Time:                        14:09:40   Log-Likelihood:            -2.3100e+05
No. Observations:               60113   AIC:                         4.620e+05
Df Residuals:                   60111   BIC:                         4.620e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     50.5494      0.133    380.936      0.000      50.289      50.810
energy         1.4439      0.198      7.297      0.000       1.056       1.832
==============================================================================
Omnibus:                     1696.941   Durbin-Watson:                   0.314
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1667.617
Skew:                           0.374   Prob(JB):                         0.00
Kurtosis:                       2.674   Cond. No.                         6.07
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [57]:
model = smf.ols(data=df_completo, formula='Pop_media ~ degree + betweenness + closeness + eigenvector ') # z1 é explicada por x1?
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              Pop_media   R-squared:                       0.012
Model:                            OLS   Adj. R-squared:                  0.011
Method:                 Least Squares   F-statistic:                     46.00
Date:                Mon, 08 Nov 2021   Prob (F-statistic):           1.75e-38
Time:                        14:09:40   Log-Likelihood:                -57935.
No. Observations:               15570   AIC:                         1.159e+05
Df Residuals:                   15565   BIC:                         1.159e+05
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      50.4032      0.099    507.583      0.000      50.209      50.598
degree       1465.6113    136.930     10.703      0.000    1197.212    1734.010
betweenness  -991.4362    349.601     -2.836      0.005   -1676.695    -306.177
closeness       0.9625      0.248      3.884      0.000       0.477       1.448
eigenvector    -7.2369     10.919     -0.663      0.507     -28.639      14.165
==============================================================================
Omnibus:                      381.418   Durbin-Watson:                   1.996
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              387.365
Skew:                           0.364   Prob(JB):                     7.67e-85
Kurtosis:                       2.739   Cond. No.                     4.62e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.62e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [58]:
model = smf.ols(data=df_recorte, formula='popularity ~ acousticness + energy + instrumentalness + liveness + loudness + speechiness + valence ') # z1 é explicada por x1?
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             popularity   R-squared:                       0.062
Model:                            OLS   Adj. R-squared:                  0.062
Method:                 Least Squares   F-statistic:                     571.3
Date:                Mon, 08 Nov 2021   Prob (F-statistic):               0.00
Time:                        14:09:40   Log-Likelihood:            -2.2909e+05
No. Observations:               60113   AIC:                         4.582e+05
Df Residuals:                   60105   BIC:                         4.583e+05
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept           65.7540      0.334    196.596      0.000      65.098      66.410
acousticness         0.0332      0.215      0.155      0.877      -0.387       0.454
energy              -7.9728      0.356    -22.418      0.000      -8.670      -7.276
instrumentalness     0.4489      0.233      1.925      0.054      -0.008       0.906
liveness            -3.9839      0.279    -14.280      0.000      -4.531      -3.437
loudness             0.7748      0.016     47.802      0.000       0.743       0.807
speechiness          6.4584      0.443     14.575      0.000       5.590       7.327
valence             -5.5402      0.191    -29.042      0.000      -5.914      -5.166
==============================================================================
Omnibus:                     1682.047   Durbin-Watson:                   0.458
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1742.131
Skew:                           0.397   Prob(JB):                         0.00
Kurtosis:                       2.747   Cond. No.                         104.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [61]:
df_recorte

,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year
5054,0.7750,[Paul Simon],0.656,303507,0.6440,0,1RxlMG0bJhJdR5H6wL2oJf,0.416000,6,0.1200,-12.878,0,She Moves On,35,1990-10-16,0.0377,120.942,0.9430,1990
5055,0.1850,[Concrete Blonde],0.275,308112,0.5980,0,30q9tmV1f3vHli0MhHtZR9,0.000011,3,0.1490,-6.757,1,"Tomorrow, Wendy",35,1990-09-16,0.0305,180.130,0.1980,1990
5057,0.9750,"[Roberta Flack, Donny Hathaway]",0.229,219600,0.0723,0,0vB3z9Jp1OFTV9gZDYkVG1,0.000176,9,0.0870,-17.460,1,For All We Know,35,1990-04-03,0.0402,79.303,0.0738,1990
5058,0.5590,[Benny Hinn],0.120,239133,0.4190,0,2w3PjEG8DcqXADd51KwqtJ,0.000000,0,0.4860,-11.881,1,Holy Spirit Thou Art Welcome,37,1990-05-12,0.0427,73.289,0.2840,1990
5059,0.3960,[Elton John],0.463,298027,0.6240,0,0EpRxgIqC79gXUMI8jPMVc,0.000013,3,0.0712,-9.174,1,You Gotta Love Someone,44,1990-11-08,0.0437,205.275,0.5470,1990
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169902,0.0227,"[Trey Songz, Summer Walker]",0.619,194576,0.7190,1,5QZ11AHm7xiytOGXGlxQi5,0.000000,0,0.0839,-4.111,1,Back Home (feat. Summer Walker),69,2020-04-29,0.1570,86.036,0.3510,2020
169903,0.2100,"[LEGADO 7, Junior H]",0.795,218501,0.5850,0,52Cpyvd2dKb6XRn313nH87,0.000001,8,0.1120,-4.451,1,Ojos De Maniaco,68,2020-02-28,0.0374,97.479,0.9340,2020
169904,0.1730,"[DripReport, Tyga]",0.875,163800,0.4430,1,4KppkflX7I3vJQk7urOJaS,0.000032,1,0.0891,-7.461,1,Skechers (feat. Tyga) - Remix,75,2020-05-15,0.1430,100.012,0.3060,2020
169907,0.0714,"[Cash Cash, Andy Grammer]",0.646,167308,0.7610,0,3wYOGJYD31sLRmBgCvWxa4,0.000000,1,0.2220,-2.557,1,I Found You,70,2020-02-28,0.0385,129.916,0.4720,2020
